In [7]:
import os
import gc
import time
from pathlib import Path
import requests
from datetime import datetime
import pandas as pd
from concurrent import futures
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing

from signals_lite.constants import DAILY_DATA_DIR, DATA_DIR
from signals_lite.data.data_utils import load_recent_data_from_file, save_daily_data, get_latest_date, save_in_folders
from credentials import EODHD_API_KEY

In [8]:
#import logging
#logging.basicConfig(level=logging.DEBUG)
#logger = logging.getLogger(__name__)

In [9]:
eodhd_api_key = EODHD_API_KEY
os.environ["EODHD_API_KEY"] = eodhd_api_key

# 1. Download raw data - save merged file

In [10]:
class StockDataDownloader:
    def __init__(self, data_dir="data", max_workers=8, eodhd_apikey:str):
        self.max_workers = max_workers
        self.data_dir = Path(data_dir)

    @staticmethod
    def load_dividends_eodhd(ticker, api_key, date_from):
        """
        Load the splits data from the EOD Historical Data API.
        """
        url = f"https://eodhistoricaldata.com/api/div/{ticker}?api_token={api_key}&fmt=json&from={date_from}"
        response = requests.get(url)

        if response.status_code == 200:
            if len(response.json()) > 0:
                res = (
                    pd.DataFrame(response.json())
                    .set_index("date")
                    .add_prefix("dividend_")
                )
                res.index = pd.to_datetime(res.index, format="%Y-%m-%d")
                res.rename(columns={"dividend_value": "dividend_amount"}, inplace=True)
                # keep only the dividend amount and date
                res = res[["dividend_amount"]]
                return res
        else:
            return None

    @staticmethod
    def load_splits_eodhd(ticker, api_key, date_from):
        """
        Load the splits data from the EOD Historical Data API.
        """
        url = f"https://eodhistoricaldata.com/api/splits/{ticker}?api_token={api_key}&fmt=json&from={date_from}"
        response = requests.get(url)

        if response.status_code == 200:
            if len(response.json()) > 0:
                df = (
                    pd.DataFrame(response.json())
                    .set_index("date")
                    .rename(columns={"split": "split_ratio"})
                )
                # parse the split ratio from string to float: '2.000000/1.000000' -> 2.0
                df["split_ratio"] = df["split_ratio"].apply(
                    lambda x: float(x.split("/")[0])
                )
                df.index = pd.to_datetime(df.index, format="%Y-%m-%d")
                return df
        else:
            return None

    @staticmethod
    def download_splits_yahoo(ticker, start_date, end_date):
        start_epoch = int(start_date.timestamp())
        end_epoch = int(end_date.timestamp())

        splits = (
            pd.read_csv(
                f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={start_epoch}&period2={end_epoch}&interval=1d&events=split&includeAdjustedClose=true"
            )
            .dropna()
            .set_index("Date")
        )

        if splits is not None and len(splits) > 1:
            splits["date64"] = pd.to_datetime(splits.index, format="%Y-%m-%d")
            splits = (
                splits.reset_index(drop=True)
                .set_index("date64")
                .sort_index()
                .rename(columns={"Stock Splits": "split_factor"})
            )

        return splits

    @staticmethod
    def download_dividends_yahoo(ticker, start_date, end_date):
        start_epoch = int(start_date.timestamp())
        end_epoch = int(end_date.timestamp())

        dividends = (
            pd.read_csv(
                f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={start_epoch}&period2={end_epoch}&interval=1d&events=div&includeAdjustedClose=true"
            )
            .dropna()
            .set_index("Date")
        )

        if dividends is not None and len(dividends) > 1:
            dividends["date64"] = pd.to_datetime(dividends.index, format="%Y-%m-%d")
            dividends = (
                dividends.reset_index(drop=True)
                .set_index("date64")
                .sort_index()
                .rename(columns={"Dividends": "dividend_amount"})
            )

        return dividends

    def yahoo_download_one(self, signals_ticker, date_from=None, date_until=None):
        if date_from is None:
            date_from = "2000-01-01"
        if date_until is None:
            date_until = datetime.today().strftime("%Y-%m-%d")

        date_from = datetime.strptime(date_from, "%Y-%m-%d")
        date_until = datetime.strptime(date_until, "%Y-%m-%d")

        start_epoch = int(date_from.timestamp())
        end_epoch = int(date_until.timestamp())

        quotes = None

        quotes = (
            pd.read_csv(
                f"https://query1.finance.yahoo.com/v7/finance/download/{signals_ticker}?period1={start_epoch}&period2={end_epoch}&interval=1d&events=history&includeAdjustedClose=true"
            )
            .dropna()
            .set_index("Date")
        )

        if quotes is not None and len(quotes) > 1:
            quotes["date64"] = pd.to_datetime(quotes.index, format="%Y-%m-%d")
            quotes = quotes.reset_index(drop=True).set_index("date64").sort_index()
            quotes.index.name = "date"
            quotes.columns = [
                "open",
                "high",
                "low",
                "close",
                "adjusted_close",
                "volume",
            ]

            dividends = StockDataDownloader.download_dividends_yahoo(
                signals_ticker, date_from, date_until
            )
            splits = StockDataDownloader.download_splits_yahoo(
                signals_ticker, date_from, date_until
            )

            if dividends is not None and len(dividends) > 1:
                quotes = quotes.join(dividends, how="left")

            if splits is not None and len(splits) > 1:
                quotes = quotes.join(splits, how="left")

        return quotes

    def eodhd_download_one(
        self, signals_ticker, api_key, date_from=None, date_until=None
    ):
        if date_from is None:
            start_date = "2000-01-01"
        else:
            start_date = date_from

        quotes = None

        r = requests.get(
            f"https://eodhistoricaldata.com/api/eod/{signals_ticker}?from={start_date}&fmt=json&api_token={api_key}"
        )

        if r.status_code == requests.codes.ok:
            if len(r.json()) > 0:
                quotes = pd.DataFrame(r.json()).set_index("date")
                quotes["date64"] = pd.to_datetime(quotes.index, format="%Y-%m-%d")
                quotes = quotes.reset_index(drop=True).set_index("date64").sort_index()
                quotes.index.name = "date"
                quotes.columns = [
                    "open",
                    "high",
                    "low",
                    "close",
                    "adjusted_close",
                    "volume",
                ]

            dividends = StockDataDownloader.load_dividends_eodhd(
                signals_ticker, api_key, date_from
            )
            splits = StockDataDownloader.load_splits_eodhd(
                signals_ticker, api_key, date_from
            )
            
            if dividends is not None:
                quotes: pd.DataFrame = quotes.join(dividends, how="left", on="date")

            if splits is not None:
                quotes: pd.DataFrame = quotes.join(splits, how="left", on="date")

        return quotes

    def download_one(self, bloomberg_ticker, map, eodhd_api_key=None, date_from=None):
        yahoo_ticker = map.loc[bloomberg_ticker, "yahoo"]
        signals_ticker = map.loc[bloomberg_ticker, "signals_ticker"]
        data_provider = map.loc[bloomberg_ticker, "data_provider"]

        if pd.isnull(signals_ticker):
            return bloomberg_ticker, None

        quotes = None
        for _ in range(3):
            try:
                if data_provider == "eodhd":
                    quotes = self.eodhd_download_one(
                        signals_ticker, eodhd_api_key, date_from=date_from
                    )
                elif data_provider == "yahoo":
                    quotes = self.yahoo_download_one(
                        signals_ticker=signals_ticker, date_from=date_from
                    )

                if quotes is not None:
                    quotes["data_provider"] = data_provider
                    
                break

            except Exception as ex:

                #logger.exception(ex)

                print(
                    f"download_one, ticker:{signals_ticker}, data provider: {data_provider}, exception:{ex}, bbg_ticker:{bloomberg_ticker}"
                )
                time.sleep(5)

        return bloomberg_ticker, quotes

    def download_all(self, ticker_map, eodhd_api_key, date_from=None):
        tickers = pd.Series(ticker_map.index).sample(frac=1).unique().tolist()
        print(f"download_all, tickers:{len(tickers)}")

        all_quotes = []

        with futures.ThreadPoolExecutor(self.max_workers) as executor:
            _futures = []
            for ticker in tqdm(tickers):
                _futures.append(
                    executor.submit(
                        self.download_one,
                        bloomberg_ticker=ticker,
                        map=ticker_map,
                        eodhd_api_key=eodhd_api_key,
                        date_from=date_from,
                    )
                )

            print(f"download_all, futures:{len(_futures)}")
            for future in tqdm(futures.as_completed(_futures), total=len(tickers)):
                bloomberg_ticker, quotes = future.result()
                if quotes is not None:
                    quotes["bloomberg_ticker"] = bloomberg_ticker
                    all_quotes.append(quotes)

        return all_quotes


In [11]:
def remove_wrong_rows(df):
    df = df[df["open"] > 0]
    df = df[df["high"] > 0]
    df = df[df["low"] > 0]
    df = df[df["close"] > 0]
    df = df[df["adjusted_close"] > 0]
    df = df[df["volume"] > 0]
    return df

def re_adjust_ohlc(df):
    ratio = df["close"] / df["adjusted_close"]
    df["open"] = df["open"] / ratio
    df["high"] = df["high"] / ratio
    df["low"] = df["low"] / ratio
    df["close"] = df["close"] / ratio
    return df

In [ ]:
def update_daily_data(data_dir:str, daily_data_dir: str):
    # read the latest date
    latest_date = get_latest_date(daily_data_dir)
    print(f"Latest date: {latest_date}")

    # download data from the latest date
    downloader = StockDataDownloader(data_dir=data_dir, max_workers=16)

    ticker_map = pd.read_csv("data/eodhd-map.csv").set_index("bloomberg_ticker")
    eodhd_api_key = os.environ["EODHD_API_KEY"]
    all_quotes = downloader.download_all(
       ticker_map, eodhd_api_key, date_from=latest_date
    )

    # save all quotes
    all_quotes = pd.concat(all_quotes)
    all_quotes = remove_wrong_rows(all_quotes)
    all_quotes = re_adjust_ohlc(all_quotes)
    save_in_folders(all_quotes, daily_data_dir)
    gc.collect()

update_daily_data(DATA_DIR, DAILY_DATA_DIR)

In [1]:
import pandas as pd

In [2]:
pd.read_parquet("../data/01_daily_adjusted/2023/08/2023-08-14.parquet")

,open,high,low,close,adjusted_close,volume,data_provider,bloomberg_ticker,dividend_amount,split_ratio,date_str
date,,,,,,,,,,,
2023-08-14,44.900,45.520,44.580,45.010,45.010,1520155,eodhd,TDC US,NaN,NaN,2023-08-14
2023-08-14,4241.000,4267.000,4217.000,4258.000,4258.000,124369,eodhd,BKG LN,NaN,NaN,2023-08-14
2023-08-14,6.005,6.063,5.987,6.054,6.054,10389032,eodhd,ENEL IM,NaN,NaN,2023-08-14
2023-08-14,513.000,513.000,502.000,510.000,510.000,114752,eodhd,5904 TT,NaN,NaN,2023-08-14
2023-08-14,154.420,155.515,153.830,155.320,155.320,1171114,eodhd,HLT US,NaN,NaN,2023-08-14
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-14,20.380,20.380,19.570,20.190,20.190,169400,eodhd,TREE US,NaN,NaN,2023-08-14
2023-08-14,7.300,7.300,7.130,7.240,7.240,1506100,eodhd,LC US,NaN,NaN,2023-08-14
2023-08-14,11.790,11.990,11.620,11.810,11.810,3056700,eodhd,VAMO3 BZ,NaN,NaN,2023-08-14
